<a href="https://colab.research.google.com/github/Gbolahan-Aziz/Recommender_Systems/blob/master/Two_Tower_Retrieval_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tensorflow-recommenders

     |████████████████████████████████| 85 kB 3.7 MB/s 


In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_recommenders as tfrs
from typing import Dict, Text
import tensorflow_datasets as tfds

In [4]:
anime = pd.read_csv('/content/drive/MyDrive/Recommender system/anime.csv')

In [5]:
ratings = pd.read_csv('/content/drive/MyDrive/Recommender system/rating.csv')

In [6]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [8]:
anime.dropna(axis=0,inplace=True)

In [9]:
anime.genre.unique()

array(['Drama, Romance, School, Supernatural',
       'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen',
       'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen',
       ..., 'Action, Comedy, Hentai, Romance, Supernatural',
       'Hentai, Sports', 'Hentai, Slice of Life'], dtype=object)

In [10]:
anime['type'].unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA'], dtype=object)

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [12]:
anime.genre = le.fit_transform(anime.genre)

In [13]:
anime.type = le.fit_transform(anime.type)

In [14]:
anime.tail()

,anime_id,name,genre,type,episodes,rating,members
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,2874,3,1,4.15,211
12290,5543,Under World,2874,3,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,2874,3,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,2874,3,1,4.98,175
12293,26081,Yasuji no Pornorama: Yacchimae!!,2874,0,1,5.46,142


In [15]:
anime.columns = ['anime_id', 'name', 'genre', 'type', 'episodes', 'anime_rating', 'members']

In [16]:
anime.head()

,anime_id,name,genre,type,episodes,anime_rating,members
0,32281,Kimi no Na wa.,2660,0,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,159,5,64,9.26,793665
2,28977,Gintama°,527,5,51,9.25,114262
3,9253,Steins;Gate,3205,5,24,9.17,673572
4,9969,Gintama&#039;,527,5,51,9.16,151266


In [17]:
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [18]:
ratings.columns = ['user_id', 'anime_id', 'user_rating']

In [19]:
ratings.replace(-1,0,inplace=True)

In [20]:
ratings.user_rating.unique()

array([ 0, 10,  8,  6,  9,  7,  3,  5,  4,  1,  2])

In [21]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   anime_id      12017 non-null  int64  
 1   name          12017 non-null  object 
 2   genre         12017 non-null  int64  
 3   type          12017 non-null  int64  
 4   episodes      12017 non-null  object 
 5   anime_rating  12017 non-null  float64
 6   members       12017 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 751.1+ KB


In [22]:
anime.episodes.value_counts()

1       5571
2       1075
12       810
13       571
26       514
        ... 
162        1
373        1
1006       1
510        1
172        1
Name: episodes, Length: 187, dtype: int64

In [23]:
(anime.episodes== 'Unknown').sum()

187

In [24]:
anime.episodes.replace('Unknown',1,inplace=True)

In [25]:
anime.episodes = anime.episodes.astype('int64')

In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column       Dtype
---  ------       -----
 0   user_id      int64
 1   anime_id     int64
 2   user_rating  int64
dtypes: int64(3)
memory usage: 178.8 MB


In [27]:
ratings.isnull().sum()

user_id        0
anime_id       0
user_rating    0
dtype: int64

In [28]:
data = pd.merge(anime,ratings,on='anime_id',how='left')

In [29]:
rating = pd.merge(anime,ratings,on='anime_id',how='right')


In [30]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7813737 entries, 0 to 7813736
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   anime_id      int64  
 1   name          object 
 2   genre         float64
 3   type          float64
 4   episodes      float64
 5   anime_rating  float64
 6   members       float64
 7   user_id       int64  
 8   user_rating   int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 596.1+ MB


In [31]:
rating.isnull().sum()

anime_id          0
name            126
genre           126
type            126
episodes        126
anime_rating    126
members         126
user_id           0
user_rating       0
dtype: int64

In [32]:
rating.dropna(axis=0,inplace=True)

In [33]:
rating.name = rating.name.astype('string')
rating.user_id = rating.user_id.astype('string')


In [34]:
# Preparing data in tf format

In [35]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(({'user_id':tf.cast(rating['user_id'].values, tf.string),
            'anime_title': tf.cast(rating['name'].values, tf.string)}
        )
    )
)

In [36]:
training_dataset

<TensorSliceDataset shapes: {user_id: (), anime_title: ()}, types: {user_id: tf.string, anime_title: tf.string}>

In [37]:
anime_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(anime['name'].values, tf.string)
    )
))

In [38]:
anime_dataset

<TensorSliceDataset shapes: (), types: tf.string>

In [39]:
training_dataset

<TensorSliceDataset shapes: {user_id: (), anime_title: ()}, types: {user_id: tf.string, anime_title: tf.string}>

In [40]:
anime_dataset

<TensorSliceDataset shapes: (), types: tf.string>

In [41]:
tf.random.set_seed(42)
shuffled = training_dataset.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [42]:
anime_titles = anime.name
user_id = rating.user_id

unique_user_id = rating.user_id.unique()
unique_anime_titles = anime.name.unique()


unique_anime_titles[:10]

array(['Kimi no Na wa.', 'Fullmetal Alchemist: Brotherhood', 'Gintama°',
       'Steins;Gate', 'Gintama&#039;',
       'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou',
       'Hunter x Hunter (2011)', 'Ginga Eiyuu Densetsu',
       'Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare',
       'Gintama&#039;: Enchousen'], dtype=object)

In [43]:
user_model = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                      vocabulary=unique_user_id, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_user_id) + 1, 32)
])                                 
                                  
anime_model = tf.keras.Sequential([
 tf.keras.layers.experimental.preprocessing.StringLookup(
     vocabulary=unique_anime_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_anime_titles) + 1, 32)                                 
])

In [44]:
anime_dataset.batch(128)

<BatchDataset shapes: (None,), types: tf.string>

In [45]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates= anime_dataset.batch(128).map(anime_model)
)
task = tfrs.tasks.Retrieval(
    metrics=metrics
)


In [46]:
# Build a model.
class Anime_Model(tfrs.Model):

  def __init__(self, user_model, anime_model):
    super().__init__()

    # Set up user representation.
    self.anime_model: tf.keras.Model = anime_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    positive_anime_embeddings = self.anime_model(features["anime_title"])

    return self.task(user_embeddings, positive_anime_embeddings)




In [47]:
model = Anime_Model(user_model, anime_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()


In [48]:
#CacheDataset shapes: {movie_title: (None,), user_id: (None,)}, types: {movie_title: tf.string, user_id: tf.string}

In [49]:
cached_train

<CacheDataset shapes: {user_id: (None,), anime_title: (None,)}, types: {user_id: tf.string, anime_title: tf.string}>

In [50]:
# Train.
model.fit(cached_train, epochs=5)


Epoch 1/5
10/10 [==============================] - 105s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0224 - factorized_top_k/top_5_categorical_accuracy: 0.1720 - factorized_top_k/top_10_categorical_accuracy: 0.2341 - factorized_top_k/top_50_categorical_accuracy: 0.2718 - factorized_top_k/top_100_categorical_accuracy: 0.2873 - loss: 70325.2536 - regularization_loss: 0.0000e+00 - total_loss: 70325.2536
Epoch 2/5
10/10 [==============================] - 103s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0405 - factorized_top_k/top_5_categorical_accuracy: 0.4290 - factorized_top_k/top_10_categorical_accuracy: 0.6216 - factorized_top_k/top_50_categorical_accuracy: 0.6630 - factorized_top_k/top_100_categorical_accuracy: 0.6692 - loss: 69310.0675 - regularization_loss: 0.0000e+00 - total_loss: 69310.0675
Epoch 3/5
10/10 [==============================] - 101s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0957 - factorized_top_k/top_5_categorical_accuracy: 

In [51]:
# Evaluate.
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 25s 5s/step - factorized_top_k/top_1_categorical_accuracy: 0.0083 - factorized_top_k/top_5_categorical_accuracy: 0.2878 - factorized_top_k/top_10_categorical_accuracy: 0.4526 - factorized_top_k/top_50_categorical_accuracy: 0.4809 - factorized_top_k/top_100_categorical_accuracy: 0.4839 - loss: 34872.0072 - regularization_loss: 0.0000e+00 - total_loss: 34872.0072


{'factorized_top_k/top_100_categorical_accuracy': 0.4839499890804291,
 'factorized_top_k/top_10_categorical_accuracy': 0.45260000228881836,
 'factorized_top_k/top_1_categorical_accuracy': 0.008299999870359898,
 'factorized_top_k/top_50_categorical_accuracy': 0.4808500111103058,
 'factorized_top_k/top_5_categorical_accuracy': 0.28780001401901245,
 'loss': 31630.052734375,
 'regularization_loss': 0,
 'total_loss': 31630.052734375}

In [58]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends anime out of the entire anime dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((anime_dataset.batch(100), anime_dataset.batch(100).map(model.anime_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Shakugan no Shana II (Second)' b'Tengen Toppa Gurren Lagann'
 b'Rosario to Vampire']


In [59]:
data = pd.read_csv('/content/drive/MyDrive/Recommender system/anime.csv')

In [60]:
data_rating = pd.read_csv('/content/drive/MyDrive/Recommender system/rating.csv')

In [68]:
user_42 = data_rating[data_rating.user_id==42]

In [72]:
user_42_df = pd.merge(data,recommended,on='anime_id',how='right')

In [85]:
  recommended = data.loc[(data['name'] == 'Shakugan no Shana II (Second)') | (data['name'] == 'Tengen Toppa Gurren Lagann') | (data['name'] == 'Rosario to Vampire')]


In [86]:
recommended

,anime_id,name,genre,type,episodes,rating,members
29,2001,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27,8.78,562962
878,2787,Shakugan no Shana II (Second),"Action, Drama, Fantasy, Romance, School, Super...",TV,24,7.79,184525
2895,2993,Rosario to Vampire,"Comedy, Ecchi, Fantasy, Harem, Romance, School...",TV,13,7.14,299278


In [87]:
user_42_df

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,143,Kannazuki no Miko,"Drama, Magic, Mecha, Romance, Shoujo Ai, Shoun...",TV,12,7.05,42927,42,9
1,269,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366,7.95,624055,42,10
2,855,Strawberry Panic,"Drama, Romance, School, Shoujo Ai",TV,26,7.45,72790,42,10
3,2964,Blue Drop: Tenshi-tachi no Gikyoku,"Drama, Sci-Fi, Shoujo Ai",TV,13,6.88,19968,42,8
4,10495,Yuru Yuri,"Comedy, School, Shoujo Ai, Slice of Life",TV,12,7.70,148140,42,7
5,14075,Zetsuen no Tempest,"Action, Drama, Fantasy, Magic, Mystery, Psycho...",TV,24,8.17,259900,42,10


In [88]:
## We can see that the recommendation system is doing quite well for a  two tower retrieval system.